In [2]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()



True

In [3]:
HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

# Authenticate
login(token=HUGGING_FACE_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/teodorrk/.cache/huggingface/token
Login successful


In [4]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from tqdm.auto import tqdm


source_model_id = "Basemodel_369M/pytorch_model_1.bin"
peft_model_id = "Convmodel_369M/adapter_model_1.bin"

In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, PeftModel

# Paths to your model and adapter files
base_model_path = "./Basemodel_369M"
conv_adapter_model = "./Convmodel_369M/adapter_model.bin"
conv_adapter_config = "./Convmodel_369M/adapter_config.json"
inst_adapter_model = "./Instmodel_369M/adapter_model.bin"
inst_adapter_config = "./Instmodel_369M/adapter_config.json"


# Load the base model and tokenizer from local files
model = AutoModelForCausalLM.from_pretrained(base_model_path, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(base_model_path, local_files_only=True)

# Load the adapter configuration directly from the local JSON file
with open(inst_adapter_config, 'r') as f:
    adapter_config_dict = json.load(f)

# Instantiate the LoraConfig object
lora_config = LoraConfig(**adapter_config_dict)

# Load the adapter model using the configuration
model = PeftModel(model, lora_config)

# Manually load the adapter weights
adapter_weights = torch.load(inst_adapter_model, map_location="cpu")
model.load_state_dict(adapter_weights, strict=False)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModel(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(64000, 1024)
        (wpe): Embedding(2048, 1024)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPT2Block(
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              

In [2]:
# Prepare input
input_text = "Forklar fourier analyse"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate text with adjusted parameters
outputs = model.generate(
    **inputs,
    max_length=200,  # Adjust maximum length of generated text
    num_return_sequences=1,  # Number of sequences to return
    #temperature=0.7,  # Sampling temperature
    #top_p=0.2,  # Top-p (nucleus) sampling
    #top_k=50,  # Top-k sampling
    do_sample=True,  # Whether to use sampling
    pad_token_id=tokenizer.eos_token_id  # Ensure pad_token_id is set
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Forklar fourier analyse og forklar hvordan dette kan virke på en gitt trekant.
